In [24]:
from pynput import keyboard
import time

import pandas as pd

cols = ['key', 'action', 'time']
data = pd.DataFrame(columns = cols)

def get_key_name(key):
    if isinstance(key, keyboard.KeyCode):
        return key.char
    else:
        return str(key)

def record_keystroke(keystroke_info):
    global data
    data = data.append(keystroke_info, ignore_index=True)

def on_press(key):
    t = time.time()
    key_name = get_key_name(key)
    record_keystroke({'key' : key_name, 'action' : 'press', 'time' : t})
#     print('Key {} pressed.'.format(key_name))

def on_release(key):
    t = time.time()
    key_name = get_key_name(key)
    record_keystroke({'key' : key_name, 'action' : 'release', 'time' : t})
#     print('Key {} released.'.format(key_name))

    if key_name == 'Key.esc':
        print('Exiting...')
        return False

with keyboard.Listener(
    on_press = on_press,
    on_release = on_release) as listener:
    listener.join()

Exiting...


In [25]:
data

,key,action,time
0,Key.enter,release,1.572949e+09
1,Key.shift_r,release,1.572949e+09
2,j,press,1.572949e+09
3,j,release,1.572949e+09
4,a,press,1.572949e+09
5,a,release,1.572949e+09
6,f,press,1.572949e+09
7,h,press,1.572949e+09
8,u,press,1.572949e+09
9,f,release,1.572949e+09
